## **TABLA DE MODELADO**

In [15]:
# DATOS
import pandas as pd

# BASE DE DATOS
from sqlalchemy import create_engine

# MODELADO
from joblib import load

**MODELOS**

In [16]:
# KMEANS
kmeans = load('C:/Users/20391117579/Dropbox/CrimeApp/Data Science Lab/Modelos/kmeans.joblib')
scaler = load('C:/Users/20391117579/Dropbox/CrimeApp/Data Science Lab/Modelos/scaler.joblib')

**TABLA FILTRADA DE SQL**

In [2]:
hostname = "localhost"
dbname = "crimewarehouse"
uname = "root"
pwd = "admin1234"

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=hostname, db=dbname, user=uname, pw=pwd))

In [4]:
def get_fct_table(query, db):
    df_fct = pd.read_sql(query, db)

    return df_fct

df_fct = get_fct_table(query="SELECT * FROM FCT_HECHOS;", db=engine)

In [5]:
df_fct.head(5)

,CONTACTO_ID,FECHA,FRANJA_HORARIA,TIPO_DELITO_KEY,LATITUD,LONGITUD,COMUNA_KEY,BARRIO_KEY
0,0,2016-01-31,12,1,-34.6398,-58.5297,9,22
1,1,2016-01-31,8,1,-34.6388,-58.5275,9,22
2,2,2016-01-31,20,1,-34.6456,-58.5265,9,22
3,3,2016-01-31,22,1,-34.6013,-58.5199,11,31
4,4,2016-01-31,8,1,-34.5896,-58.5187,11,31


**TABLA DE MODELADO**

In [12]:
def transform_time_series(df_fct):
    # Serie temporal
    df_fct['FECHA'] = pd.to_datetime(df_fct['FECHA'])
    df_fct['MES'] = df_fct['FECHA'].dt.month 
    df_fct['DIADELMES'] = df_fct['FECHA'].dt.day  
    df_fct['DIADELAÑO'] = df_fct['FECHA'].dt.day_of_year
    df_fct['SEMANADELAÑO'] = df_fct['FECHA'].dt.isocalendar().week.astype(int)
    df_fct['TRIMESTRE'] = df_fct['FECHA'].dt.quarter  
    df_fct['DIADELASEMANA'] = df_fct['FECHA'].dt.dayofweek 

    # Estacion, Fin de semana
    df_fct['ESFINDESEMANA'] = df_fct['DIADELASEMANA'].apply(lambda x: 1 if x in [5, 6] else 0)
    df_fct['ESTACION'] = df_fct['MES'].apply(lambda x: 4 if x in [12, 1, 2] else (1 if x in [3, 4, 5] else (2 if x in [6, 7, 8] else 3)))

    # Horario Categorizado
    bins = [0, 6, 12, 18, 24]
    labels = [1, 2, 3, 4]
    df_fct['HORARIO_CATEGORIZADO'] = pd.cut(df_fct['FRANJA_HORARIA'], bins=bins, labels=labels, right=False).astype(int)

    df_fct_time_series = df_fct[['FECHA', 'LATITUD', 'LONGITUD', 'ESTACION', 'MES', 'TRIMESTRE',
                                'SEMANADELAÑO', 'DIADELAÑO', 'DIADELMES', 'DIADELASEMANA',
                                'ESFINDESEMANA', 'HORARIO_CATEGORIZADO', 'FRANJA_HORARIA']]

    return df_fct_time_series

In [13]:
df_fct_time_series = transform_time_series(df_fct)